In [ ]:
!wget https://hf.co/danjacobellis/walloc/resolve/main/Stereo_Li_96c_J8_nf8_v1.0.2.pth

In [ ]:
!python3 -m pip install -U git+https://github.com/facebookresearch/demucs#egg=demucs

In [2]:
import torch
import numpy as np
import torchaudio
import tempfile
from cdpam import CDPAM
from demucs.separate import Separator
from IPython.display import Audio
from datasets import load_dataset, Dataset
from walloc import walloc
from spauq.core.metrics import spauq_eval
from fastprogress.fastprogress import progress_bar
class Config: pass

In [3]:
device = "cuda"
checkpoint = torch.load("Stereo_Li_96c_J8_nf8_v1.0.2.pth",map_location="cpu",weights_only=False)
codec_config = checkpoint['config']
codec = walloc.Codec1D(
    channels = codec_config.channels,
    J = codec_config.J,
    Ne = codec_config.Ne,
    Nd = codec_config.Nd,
    latent_dim = codec_config.latent_dim,
    latent_bits = codec_config.latent_bits,
    lightweight_encode = codec_config.lightweight_encode,
    post_filter = codec_config.post_filter
).to(device)
codec.load_state_dict(checkpoint['model_state_dict'])
codec.eval();

/home/dgj335/.local/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


In [4]:
device = "cuda"
separator = Separator()
cdpam_loss = CDPAM()
MUSDB = load_dataset("danjacobellis/musdb18HQ", split='validation')

/home/dgj335/.local/lib/python3.10/site-packages/cdpam/cdpam.py:45: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(modfolder,map_location="cpu")['state']


Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/22 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/22 [00:00<?, ?it/s]

In [5]:
separator = Separator()

In [6]:
def pad(audio, p=2**16):
    B,C,L = audio.shape
    padding_size = (p - (L % p)) % p
    if padding_size > 0:
        audio = torch.nn.functional.pad(audio, (0, padding_size), mode='constant', value=0)
    return audio

In [7]:
max_duration=44100*180
SSDR = []
SRDR = []
PSNR = []
CDPAM_dB = []
res_red = torchaudio.transforms.Resample(44100, 16536).to(device)
res_inc = torchaudio.transforms.Resample(16536, 44100).to(device)
with torch.no_grad():
    for i_sample in progress_bar(range(0, len(MUSDB), 5)):
        SSDR.append([])
        SRDR.append([])
        PSNR.append([])
        CDPAM_dB.append([])
        y = []
        for i_instr in range(5):
            sample = MUSDB[i_sample+i_instr]
            instr = sample['instrument']
            x, fs = torchaudio.load(sample['audio']['bytes'])
            x = x[:,:max_duration]
            L = x.shape[-1]
            x_padded = pad(x.unsqueeze(0), 2**16).to(device)

            X = codec.wavelet_analysis(x_padded,codec.J)
            Y = codec.encoder(X)
            X_hat = codec.decoder(Y)
            x_hat = codec.wavelet_synthesis(X_hat,codec.J)
            x_hat = codec.post(x_hat)
                              
            if i_instr == 0:
                pred = separator.separate_tensor(x_hat[0])
                
            else:
                ℓ = min(x_padded.shape[-1], pred[1][instr].shape[-1])
                SDR = spauq_eval(
                    reference=x_padded[0,:,:ℓ].to("cpu"),
                    estimate=pred[1][instr][:,:ℓ].to("cpu"),
                    fs = fs
                )
                psnr = 20*np.log10(2) - 10*np.log10(torch.nn.functional.mse_loss(
                    x_padded[0,:,:ℓ],
                    pred[1][instr][:,:ℓ]
                ).item())
                cdpam = cdpam_loss.forward(
                    wav_in=x_padded[0,:,:ℓ],
                    wav_out=pred[1][instr][:,:ℓ]
                )
                SSDR[-1].append(SDR['SSR'])
                SRDR[-1].append(SDR['SRR'])
                PSNR[-1].append(psnr)
                CDPAM_dB[-1].append(-np.log10(cdpam.mean().item()))
SSDR = torch.tensor(SSDR)
SRDR = torch.tensor(SRDR)
PSNR = torch.tensor(PSNR)
CDPAM_dB = torch.tensor(CDPAM_dB)

/home/dgj335/.local/lib/python3.10/site-packages/spauq/core/preprocessing.py:325: UserWarning: No forgive_mode specified, defaulting to `none`
  warnings.warn(


In [8]:
metrics = {
    'SSDR': SSDR,
    'SRDR': SRDR,
    'PSNR': PSNR,
    'CDPAM_dB': CDPAM_dB,
}

instruments = ['other', 'drums', 'bass', 'vocals']

data = {}
for metric_name, metric_tensor in metrics.items():
    for i, instr in enumerate(instruments):
        column_name = f"{metric_name}_{instr}"
        data[column_name] = metric_tensor[:, i].tolist()
dataset = Dataset.from_dict(data)

In [9]:
dataset.push_to_hub("danjacobellis/LSDIR_demucs_5x_walloc",split='validation')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/danjacobellis/LSDIR_demucs_5x_walloc/commit/1835a584eeaca8e6df2e646bc69e43a09e8f3080', commit_message='Upload dataset', commit_description='', oid='1835a584eeaca8e6df2e646bc69e43a09e8f3080', pr_url=None, pr_revision=None, pr_num=None)

In [10]:
dataset.to_pandas().mean(axis=0)

SSDR_other         14.389502
SSDR_drums         16.241770
SSDR_bass          15.289354
SSDR_vocals        11.135118
SRDR_other          4.027712
SRDR_drums          6.713292
SRDR_bass          -0.135855
SRDR_vocals        -2.275065
PSNR_other         36.374665
PSNR_drums         37.858339
PSNR_bass          40.890640
PSNR_vocals        39.673793
CDPAM_dB_other      4.734882
CDPAM_dB_drums      4.175864
CDPAM_dB_bass       4.946015
CDPAM_dB_vocals     4.616613
dtype: float64

In [11]:
dataset.to_pandas()

,SSDR_other,SSDR_drums,SSDR_bass,SSDR_vocals,SRDR_other,SRDR_drums,SRDR_bass,SRDR_vocals,PSNR_other,PSNR_drums,PSNR_bass,PSNR_vocals,CDPAM_dB_other,CDPAM_dB_drums,CDPAM_dB_bass,CDPAM_dB_vocals
0,12.648648,25.376416,22.155269,6.871884,3.584051,11.527631,10.170296,0.856013,37.565988,39.689890,41.101903,39.806242,3.144934,4.148777,5.132314,4.522501
1,16.993430,12.608602,21.592562,16.748692,6.167540,4.233242,10.504444,7.885015,35.604452,36.768817,43.123180,38.786025,4.778637,3.627605,5.375834,4.337453
2,19.265118,8.825354,11.098491,6.770587,7.151326,2.273646,5.102899,-0.790488,34.519905,38.721591,38.233524,37.648615,4.979984,4.142036,4.566220,4.809476
3,13.073877,19.031239,11.864889,0.013766,4.138247,8.373578,4.844052,-28.540757,34.489961,36.594117,34.869249,40.128373,4.380311,4.273608,4.586163,4.816226
4,17.432915,0.838355,12.639307,6.024648,6.047601,-5.275952,6.345375,0.754285,41.529964,44.543844,42.931966,47.914480,5.083652,4.926868,5.253874,5.081591
5,12.902636,24.205283,27.472969,0.000000,4.453315,12.496213,16.097304,-80.000000,39.429845,41.368199,41.486147,51.613883,4.330050,4.401863,5.111062,5.318121
6,18.548949,22.360384,8.216927,10.214440,5.728244,9.632592,3.860319,3.790145,34.840063,40.563248,43.013770,36.357868,4.940490,4.394013,5.098688,4.321275
7,26.829587,10.526079,22.093393,3.995676,9.683105,4.364414,10.805436,-3.884071,35.672760,42.427580,44.034664,36.921141,5.085008,4.449695,4.191931,4.458590
8,9.566623,8.500108,22.394645,14.295396,1.094883,2.159550,11.579537,5.603527,34.895392,36.891169,41.571123,34.946505,4.549949,3.975128,5.156226,4.514801
9,17.770707,19.002764,6.068189,8.514266,6.356944,8.745703,1.661973,2.547568,35.179447,39.160760,38.538198,36.800435,5.192478,4.090090,4.608984,4.485213
